# Day 8: Treetop Tree House

The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these trees as a reforestation effort. Now, they're curious if this would be a good location for a tree house.

First, determine whether there is enough tree cover here to keep a tree house hidden. To do this, you need to count the number of trees that are visible from outside the grid when looking directly along a row or column.

The Elves have already launched a quadcopter to generate a map with the height of each tree (your puzzle input). For example:
```
30373
25512
65332
33549
35390
```
Each tree is represented as a single digit whose value is its height, where 0 is the shortest and 9 is the tallest.

A tree is visible if all of the other trees between it and an edge of the grid are shorter than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.

All of the trees around the edge of the grid are visible - since they are already on the edge, there are no trees to block the view. In this example, that only leaves the interior nine trees to consider:

- The top-left 5 is visible from the left and top. (It isn't visible from the right or bottom since other trees of height 5 are in the way.)
- The top-middle 5 is visible from the top and right.
- The top-right 1 is not visible from any direction; for it to be visible, there would need to only be trees of height 0 between it and an edge.
- The left-middle 5 is visible, but only from the right.
- The center 3 is not visible from any direction; for it to be visible, there would need to be only trees of at most height 2 between it and an edge.
- The right-middle 3 is visible from the right.
- In the bottom row, the middle 5 is visible, but the 3 and 4 are not.

With 16 trees visible on the edge and another 5 visible in the interior, a total of 21 trees are visible in this arrangement.

Consider your map; how many trees are visible from outside the grid?

In [64]:
testData = """30373
25512
65332
33549
35390"""

testAnswer = 21


## Approach

We need work out how many trees are visible. Don't double count trees.

Have a 2D array for heights. Convert this into a 2D array of visibily (initially false, set to true if visible)

Basic implementation, scan every tree from every direction.
Oooh... but if we iterate through trees, we can stop whenever we get to a visible value - might cut down a fair bit of compute.

But might be easier, and neater, to just scan the forest from each side... the we can store heights as we go.


But first, we need to parse the input into a 2D array


In [71]:
def parseInput(input:str)->[[int]]:
    treeheights = []
    for line in input.splitlines():
        rowInts = [int(k) for k in line]
        treeheights.append(rowInts)
    return treeheights


def print2DArray(input:[[]]):
    print('-'*len(input[0]*3))
    for r in input:
        print(str(r))
    print('-'*len(input[0]*3))

#test
testTrees = parseInput(testData)
print2DArray(testTrees)




---------------
[3, 0, 3, 7, 3]
[2, 5, 5, 1, 2]
[6, 5, 3, 3, 2]
[3, 3, 5, 4, 9]
[3, 5, 3, 9, 0]
---------------


In [72]:
def scanForest(forest:[[int]])->[[bool]]:
    #create empty array for visibilty
    rows = len(forest)
    cols = len(forest[0])
    treeVis = [ [False]*cols for i in range(rows)]

    #Scan from Left & Right hand edges
    for r in range(rows):
        #Scan from left
        highestTree = -1 #we need to start with a negative number, because a tree of zero height is still visible on the edge
        for c in range(cols):
            #test tree and update visibitily
            treeHeight = forest[r][c]
            if treeHeight > highestTree:
                treeVis[r][c] = True
                highestTree = treeHeight
        #Scan from right
        highestTree = -1
        for c in range(cols-1,-1,-1):
            treeHeight = forest[r][c]
            if treeHeight > highestTree:
                treeVis[r][c] = True
                highestTree = treeHeight



    #Also need to Scan from Top & Bottom
    for c in range(cols):
        highestTree = -1 #we need to start with a negative number, because a tree of zero height is still visible on the edge
        for r in range(rows):
            #test tree and update visibitily
            treeHeight = forest[r][c]
            if treeHeight > highestTree:
                treeVis[r][c] = True
                highestTree = treeHeight
            
        highestTree = -1
        for r in range(rows-1,-1,-1):
            treeHeight = forest[r][c]
            if treeHeight > highestTree:
                treeVis[r][c] = True
                highestTree = treeHeight

    return treeVis

#test
testVis = scanForest(testTrees)
print2DArray(testVis)

---------------
[True, True, True, True, True]
[True, True, True, False, True]
[True, True, False, True, True]
[True, False, True, False, True]
[True, True, True, True, True]
---------------


In [73]:
def countVisibleTrees(treeVis:[[bool]])->int:
    count = 0
    for r in treeVis:
        for c in r:
            if c==True:
                count += 1
    return count

#test
count = countVisibleTrees(testVis)
print(count) # should be 21
print(count==21)


21
True


In [74]:
#Run with real input
input = open("day8input.txt").read()
realForest = parseInput(input)
realVis = scanForest(realForest)
print(countVisibleTrees(realVis))


1794


# Part Two

Content with the amount of tree cover available, the Elves just need to know the best spot to build their tree house: they would like to be able to see a lot of trees.

To measure the viewing distance from a given tree, look up, down, left, and right from that tree; stop if you reach an edge or at the first tree that is the same height or taller than the tree under consideration. (If a tree is right on the edge, at least one of its viewing distances will be zero.)

The Elves don't care about distant trees taller than those found by the rules above; the proposed tree house has large eaves to keep it dry, so they wouldn't be able to see higher than the tree house anyway.

In the example above, consider the middle 5 in the second row:
```
30373
25512
65332
33549
35390
```

- Looking up, its view is not blocked; it can see 1 tree (of height 3).
- Looking left, its view is blocked immediately; it can see only 1 tree (of height 5, right next to it).
- Looking right, its view is not blocked; it can see 2 trees.
- Looking down, its view is blocked eventually; it can see 2 trees (one of height 3, then the tree of height 5 that blocks its view).

A tree's scenic score is found by multiplying together its viewing distance in each of the four directions. For this tree, this is 4 (found by multiplying 1 * 1 * 2 * 2).

However, you can do even better: consider the tree of height 5 in the middle of the fourth row:
```
30373
25512
65332
33549
35390
```
- Looking up, its view is blocked at 2 trees (by another tree with a height of 5).
- Looking left, its view is not blocked; it can see 2 trees.
- Looking down, its view is also not blocked; it can see 1 tree.
- Looking right, its view is blocked at 2 trees (by a massive tree of height 9).

This tree's scenic score is 8 (2 * 2 * 1 * 2); this is the ideal spot for the tree house.

Consider each tree on your map. What is the highest scenic score possible for any tree?

In [69]:
def maxScenic(forest:[[int]])->int:
    maxScore = 0
    
    rows = len(forest)
    cols = len(forest[0])

    for r in range(rows):
        for c in range(cols):
            #scanning tree by tree
            treeHeight = forest[r][c]
            #look right
            rightVis = 0
            for cx in range(c+1,cols,1):
                rightVis += 1
                xHeight = forest[r][cx]
                if xHeight >= treeHeight:
                    break
            #look left
            leftVis = 0
            for cx in range(c-1,-1,-1):
                leftVis += 1
                xHeight = forest[r][cx]
                if xHeight >= treeHeight:
                    break
            #look down
            downVis = 0
            for rx in range(r+1,rows,1):
                downVis += 1
                xHeight = forest[rx][c]
                if xHeight >= treeHeight:
                    break
            #look up
            upVis = 0
            for rx in range(r-1,-1,-1):
                upVis += 1
                xHeight = forest[rx][c]
                if xHeight >= treeHeight:
                    break
            
            #calculate scenic score
            scenicScore = rightVis * leftVis * upVis * downVis
            print(str(r) + '-' + str(c) + ': ' + str(scenicScore) + ' //' + str(leftVis) + ':' + str(rightVis) + ':' + str(upVis) + ':' + str(downVis))
            if scenicScore > maxScore:
                maxScore = scenicScore

    return maxScore

#test
#printArray(testTrees)
testScenic = maxScenic(testTrees)
print(testScenic)
#should be 8
print(testScenic==8)

0-0: 0 //0:2:0:2
0-1: 0 //1:1:0:1
0-2: 0 //2:1:0:1
0-3: 0 //3:1:0:4
0-4: 0 //1:0:0:3
1-0: 0 //0:1:1:1
1-1: 1 //1:1:1:1
1-2: 4 //1:2:1:2
1-3: 1 //1:1:1:1
1-4: 0 //2:0:1:1
2-0: 0 //0:4:2:2
2-1: 6 //1:3:1:2
2-2: 1 //1:1:1:1
2-3: 2 //1:1:2:1
2-4: 0 //1:0:1:1
3-0: 0 //0:1:1:1
3-1: 1 //1:1:1:1
3-2: 8 //2:2:2:1
3-3: 3 //1:1:3:1
3-4: 0 //4:0:3:1
4-0: 0 //0:1:1:0
4-1: 0 //1:2:2:0
4-2: 0 //1:1:1:0
4-3: 0 //3:1:4:0
4-4: 0 //1:0:1:0
8
True


In [70]:
#real run
realScenic = maxScenic(realForest)
print(realScenic)


0-0: 0 //0:3:0:3
0-1: 0 //1:1:0:1
0-2: 0 //1:1:0:1
0-3: 0 //3:2:0:6
0-4: 0 //1:1:0:1
0-5: 0 //2:1:0:2
0-6: 0 //1:1:0:2
0-7: 0 //1:1:0:3
0-8: 0 //1:2:0:1
0-9: 0 //1:1:0:1
0-10: 0 //2:1:0:2
0-11: 0 //1:2:0:1
0-12: 0 //1:1:0:2
0-13: 0 //13:5:0:2
0-14: 0 //1:1:0:1
0-15: 0 //2:3:0:2
0-16: 0 //1:1:0:1
0-17: 0 //2:1:0:1
0-18: 0 //5:1:0:2
0-19: 0 //1:3:0:3
0-20: 0 //1:1:0:1
0-21: 0 //2:1:0:1
0-22: 0 //22:1:0:12
0-23: 0 //1:3:0:8
0-24: 0 //1:2:0:2
0-25: 0 //1:1:0:1
0-26: 0 //3:9:0:1
0-27: 0 //1:1:0:1
0-28: 0 //2:2:0:1
0-29: 0 //1:1:0:1
0-30: 0 //2:1:0:1
0-31: 0 //1:2:0:1
0-32: 0 //1:1:0:1
0-33: 0 //2:2:0:3
0-34: 0 //1:1:0:2
0-35: 0 //9:9:0:2
0-36: 0 //1:1:0:2
0-37: 0 //2:1:0:2
0-38: 0 //1:3:0:2
0-39: 0 //1:2:0:1
0-40: 0 //1:1:0:1
0-41: 0 //3:3:0:1
0-42: 0 //1:1:0:1
0-43: 0 //1:1:0:2
0-44: 0 //9:1:0:4
0-45: 0 //1:1:0:3
0-46: 0 //1:6:0:1
0-47: 0 //1:5:0:3
0-48: 0 //1:1:0:1
0-49: 0 //1:1:0:2
0-50: 0 //1:2:0:1
0-51: 0 //1:1:0:1
0-52: 0 //52:46:0:3
0-53: 0 //1:1:0:1
0-54: 0 //1:1:0:2
0-55: 0 //1:1:0